In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

In [2]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "dailyrainfall/"

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  
files = data["files"] 
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [9]:
# %%time

# files_to_dl = ["data.zip"]  

# for file in files:
#     if file["name"] in files_to_dl:
#         os.makedirs(output_directory, exist_ok=True)
#         urlretrieve(file["download_url"], output_directory + file["name"])

In [10]:
# %%time
# with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
#     f.extractall(output_directory)


In [11]:
%ls -ltr dailyrainfall/

total 10703464
drwxr-xr-x  30 Jialing  staff        960 27 Mar 14:23 __MACOSX/
-rw-r--r--   1 Jialing  staff   95376895 27 Mar 15:07 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
-rw-r--r--   1 Jialing  staff   94960113 27 Mar 15:07 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 Jialing  staff   82474546 27 Mar 15:07 NorESM2-LM_daily_rainfall_NSW.csv
-rw-r--r--   1 Jialing  staff  127613760 27 Mar 15:07 ACCESS-CM2_daily_rainfall_NSW.csv
-rw-r--r--   1 Jialing  staff  232118894 27 Mar 15:07 FGOALS-f3-L_daily_rainfall_NSW.csv
-rw-r--r--   1 Jialing  staff  330360682 27 Mar 15:07 CMCC-CM2-HR4_daily_rainfall_NSW.csv
-rw-r--r--   1 Jialing  staff  254009247 27 Mar 15:07 MRI-ESM2-0_daily_rainfall_NSW.csv
-rw-r--r--   1 Jialing  staff  235661418 27 Mar 15:07 GFDL-CM4_daily_rainfall_NSW.csv
-rw-r--r--   1 Jialing  staff  294260911 27 Mar 15:07 BCC-CSM2-MR_daily_rainfall_NSW.csv
-rw-r--r--   1 Jialing  staff  295768615 27 Mar 15:07 EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 Jialing  

In [15]:
%%time
## here we are using a normal python way for merging the data 
import pandas as pd
files = glob.glob('dailyrainfall/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall("/([^_]*)", file)[0])
                for file in files)
              )
df.to_csv("combined_data.csv")

CPU times: user 3min, sys: 9.42 s, total: 3min 10s
Wall time: 3min 14s


In [16]:
df.tail()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
2014-12-27 12:00:00,-30.157068,-29.21466,153.125,154.375,6.689683,SAM0-UNICON
2014-12-28 12:00:00,-30.157068,-29.21466,153.125,154.375,7.862555,SAM0-UNICON
2014-12-29 12:00:00,-30.157068,-29.21466,153.125,154.375,10.005026,SAM0-UNICON
2014-12-30 12:00:00,-30.157068,-29.21466,153.125,154.375,8.541592,SAM0-UNICON
2014-12-31 12:00:00,-30.157068,-29.21466,153.125,154.375,68.117489,SAM0-UNICON


In [17]:
df.shape

(62467843, 6)

In [18]:
%%time
df_1 = pd.read_csv("combined_data.csv")
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
CPU times: user 26.8 s, sys: 6.36 s, total: 33.1 s
Wall time: 36.2 s


In [19]:
%%time
use_cols = ['model']
df_2 = pd.read_csv("combined_data.csv",usecols=use_cols)
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
CPU times: user 14.4 s, sys: 1.14 s, total: 15.6 s
Wall time: 15.9 s


In [20]:
%%time
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("combined_data.csv", chunksize=10_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int64
CPU times: user 27.5 s, sys: 4.37 s, total: 31.9 s
Wall time: 33.6 s


In [21]:
print(f"Memory usage with float64: {df.memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {df.astype('float32', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 3498.20 MB
Memory usage with float32: 2248.84 MB


In [46]:
filepathcsv = "/Users/Jialing/525/lab1/combined_data.csv"
filepathparquet = "/Users/Jialing/525/lab1/combined_data.parquet"
filepathparquetr = "/Users/Jialing/525/lab1/combined_data_r.parquet"

In [47]:
import os
os.environ['R_HOME'] = '/Users/Jialing/opt/miniconda3/envs/525_2023/lib/R'

In [48]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [49]:
import pyarrow.dataset as ds
import pyarrow as pa
import pandas as pd
import pyarrow 
from pyarrow import csv
import rpy2_arrow.pyarrow_rarrow as pyra

In [50]:
%%time
dataset = ds.dataset(filepathcsv, format="csv")

table = dataset.to_table()

r_table = pyra.converter.py2rpy(table)

CPU times: user 10.9 s, sys: 1.36 s, total: 12.3 s
Wall time: 11.5 s


In [53]:
%%time
%%R -i r_table

suppressMessages(library(dplyr))
result <- r_table |> count(model) |> collect()
print(result)

# A tibble: 27 × 2
   model                  n
   <chr>              <int>
 1 MPI-ESM-1-2-HAM   966420
 2 AWI-ESM-1-1-LR    966420
 3 NorESM2-LM        919800
 4 ACCESS-CM2       1932840
 5 FGOALS-f3-L      3219300
 6 CMCC-CM2-HR4     3541230
 7 MRI-ESM2-0       3037320
 8 GFDL-CM4         3219300
 9 BCC-CSM2-MR      3035340
10 EC-Earth3-Veg-LR 3037320
# ℹ 17 more rows
# ℹ Use `print(n = ...)` to see more rows
CPU times: user 1.15 s, sys: 567 ms, total: 1.72 s
Wall time: 387 ms
